In [1]:
import sys
sys.path.append('../')
import graph.random_graph as random_graph
import util.create_population as create_population
import core_nn.pure_fit_model as pure_fit_model

import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
import random

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
class general_EPIs_dense(torch.nn.Module):
    def __init__(self, contact, psMatrices, crossImmMatrix, train=False, sampleAsState=False, device="cpu"):
        super(general_EPIs_dense, self).__init__()
        self._n= contact.size()[0]
        self._k= psMatrices.size()[0]
        self._contact= contact
        self._psMatrices= psMatrices
        self._crossImmMatrix= crossImmMatrix
        self._train= train
        self._device= device
        self._sampleAsState= sampleAsState
        self._Linear= self.encode_linear()
        if train==True:
            self._Linear=torch.nn.Parameter(self._Linear)
        self._init_mask_T()
    def forward(self, state): # state shape:(K, n, 4, 1)
        A= self._Linear.matmul(state[:, None, :, None, ...])
        Psts= torch.prod(A, dim=0)
        Psts= Psts.squeeze()
        sumPsts= Psts.sum(dim= -2)+ 1e-8
        sumPsts= sumPsts[:, :, None, :]
        Psts= Psts/sumPsts
        Psts2= Psts[:, :, 1, 0]
        s= state[:, :, 0:1, 0]
        i= state[:, :, 2:3, 0]
        s= s.permute(0, 2, 1)
        Temp= self._contact*i*s
        Psts2= 1-Temp*Psts2 [..., None]
        Psts2= 1-torch.prod(Psts2, dim=-2)
        Psts[:, :, 1, 0]= Psts2
        Psts[:, :, 0, 0]= 1- Psts2
        state= state.squeeze()
        newState= Psts.matmul(state[..., None])# new state shape:(K, n, 4, 1)
        return newState
        
    def _init_mask_T(self):
        self._mask1= torch.tensor([ [0, 0, 0, 0],
                                    [0, 1, 1, 0],
                                    [0, 0, 1, 1],
                                    [0, 0, 0, 1]], device=self._device) 
        self._mask2= torch.tensor([ [0, 1, 0, 0],
                                    [0, 0, 0, 0],
                                    [0, 0, 0, 0],
                                    [0, 0, 0, 0]], device=self._device) 
        self._T    = torch.tensor([ [0, 0, 0, 0],
                                    [1, 0, 0, 0],
                                    [0, 0, 0, 0],
                                    [0, 0, 0, 0]], device=self._device) 
        self._mask1= self._mask1[None, ..., None]
    def encode_linear(self):
        ts= 4
        linear= torch.ones(self._n, self._n, 1, ts, ts, ts, device=self._device)
        for i in range(self._n): # k layer
            for j in range(self._n):# prod layer
                if i==j: #load paMatrices
                    for k in range(ts):
                        linear[j, i, 0, :, :, k]= self._psMatrices[i, ...].T
                else: #load crossImmMatrix
                    linear[j, i, 0, 1, 0, 2:4]= self._crossImmMatrix[j, i]
        return linear


In [ ]:
def sim(timeHorizon, model: general_EPIs_dense, state, device="cpu"):
    
    pass

def new_case_behaviour(state:torch.tensor):
    tempN= state.shape[1]
    tempkn= state.shape[0]
    sumState= state.sum(dim=1)
    for i in range(tempkn):
        if sumState[i, 0, 0]<tempN-0.5:
            firstInfected= random.randint(0, n-1)
            state[i, firstInfected, 2, 0]= 1
            state[i, firstInfected, 0, 0]= 0
            return tempkn-i-1 # left loop
    return 0

ps=  torch.tensor([[0.75, 0.25, 0, 0],
                    [0, 0.6, 0.4, 0],
                    [0, 0, 0.95, 0.05],
                    [0, 0, 0, 1]], device=device)

psMtrices= torch.stack([ps, ps, ps])

crossImmMatrix= torch.tensor([[0, 0.2, 0.2],
                              [0.2, 0, 0.2],
                              [0.2, 0.2, 0]], device=device)
n= 1000
avgDegree= 6
timeHorizon= 400
Kn= 3 
state= create_population.general_populations(Kn, n, device=device)# 3, 1000, 4, 1
new_case_behaviour(state)
contact=random_graph.get_ER_random_contact(n, avgDegree, device=device)
contact= contact[None, ...]
inferenceModel= general_EPIs_dense(contact, psMtrices, crossImmMatrix, device=device)
